In [ ]:
import pandas as pd
try:
    from sentence_transformers import SentenceTransformer
except:
    !pip install sentence_transformers
    from sentence_transformers import SentenceTransformer
import faiss

In [ ]:
df = pd.read_csv("../data/cleaned_data.csv")

In [ ]:
#confirm that the number of lines is equal to the number of rows in the dataframe
print(df.shape[0] == len(lines))
df = df.drop_duplicates()
df['Role'].unique()
roles_to_keep = ['Frontend Web Developer', 'Wireless Network Engineer', 'User Interface Designer', 'Front-End Developer', 'IT Support Specialist', 'Database Developer', 'Social Science Researcher', 'Record Keeper',
       'Staff Nurse', 'International Tax Consultant', 'Risk Analyst',
       'Speech Pathologist', 'Regional Sales Director',
       'Mental Health Counselor', 'Urban Planner',
       'Strategic Account Manager', 'User Researcher',
       'Financial Analyst', 'Desktop Support Technician' ]

df = df[df['Role'].isin(roles_to_keep)]
df['Role'].unique()
df.shape

df

#(1610462, 7)
#shape after filtering to keep only the roles we are interested in
#(86322, 7)
lines = []
for index, row in df.iterrows():
    row_string = " ".join(row.astype(str))  # Convert each value to string and join
    lines.append(row_string)

print(lines[0])


In [ ]:
model = SentenceTransformer('paraphrase-MiniLM-L6-v2')
embeddings = model.encode(lines, show_progress_bar=True)
embeddings.shape

In [ ]:
d = embeddings.shape[1]
index = faiss.IndexFlatL2(d)
index.add(embeddings)
index.is_trained


In [ ]:
k = 4
query = "I am a software engineer with 5 years of experience"
query_embedding = model.encode(query)
query_embedding = query_embedding.reshape(1, -1)

In [ ]:
D, I = index.search(query_embedding, k)
indices = I[0]
upper_bound = df.shape[0]

for index in indices:
    if index < upper_bound:
        print(df.iloc[index], '\n\n')
    else:
        print("No match found")